<a href="https://colab.research.google.com/github/young-hyun-park/capston_design/blob/main/inceptionv3_patch_size_224_random_11_multilabelclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.io import decode_image, read_file
from torchsummary import summary
from PIL import Image
import os
import re
import nibabel as nib

## Data loading

In [ ]:
zip_path = sorted(glob('/content/drive/Shareddrives/캡스톤 디자인1/dataset/original_512/*'))

In [ ]:
import zipfile
for path in zip_path:
  zip_file = zipfile.ZipFile(path)
  zip_file.extractall('/content/datasets/'+path.split('/')[-1].split('.')[0])
  zip_file.close()

In [ ]:
my_glob = glob('/content/datasets/images*/*.png')
print('Number of Observations: ', len(my_glob))

Number of Observations:  76666


In [ ]:
df = pd.read_csv('/content/drive/Shareddrives/캡스톤 디자인1/NIH_Dataframe.csv')

In [ ]:
full_img_paths = {os.path.basename(x): x for x in my_glob}
df['full_path'] = df['img_ind'].map(full_img_paths.get)

In [ ]:
df

,Cardiomegaly,Hernia,Infiltration,Nodule,Emphysema,Effusion,Atelectasis,Pleural_Thickening,Pneumothorax,Mass,Fibrosis,Consolidation,Edema,Pneumonia,img_ind,full_path
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00000001_000.png,/content/datasets/images_01/00000001_000.png
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00000001_001.png,/content/datasets/images_01/00000001_001.png
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00000001_002.png,/content/datasets/images_01/00000001_002.png
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00000003_000.png,/content/datasets/images_01/00000003_000.png
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00000003_001.png,/content/datasets/images_01/00000003_001.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51754,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,00030786_007.png,/content/datasets/images_12/00030786_007.png
51755,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00030789_000.png,/content/datasets/images_12/00030789_000.png
51756,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,00030793_000.png,/content/datasets/images_12/00030793_000.png
51757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,00030795_000.png,/content/datasets/images_12/00030795_000.png


In [ ]:
df_for_col = pd.read_csv('/content/drive/Shareddrives/캡스톤 디자인1/NIH_Dataframe.csv')
target_cols = df_for_col.drop(['img_ind'], axis=1).columns.to_list()
n_classes = len(target_cols)

In [ ]:
n_classes

14

In [ ]:
from sklearn.model_selection import train_test_split
paths = df['full_path'].values
columns = df.columns[:-2]
labels = df[columns].values
(train_path, val_path ,train_labels , val_labels) = train_test_split(paths,labels,test_size = 0.2,random_state = 11)
(train_path, tuning_path, train_labels, tuning_labels) = train_test_split(train_path,train_labels,test_size = 0.12,random_state = 11)

In [ ]:
#pip install -U albumentations

In [ ]:
!pip install --upgrade albumentations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 102 kB 13.4 MB/s 
     |████████████████████████████████| 47.8 MB 1.2 MB/s 
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:
#!pip uninstall opencv-python-headless

In [ ]:
#!pip install opencv-python-headless==4.1.2.30

In [ ]:
!pip install --upgrade opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60.5 MB 88 kB/s 
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30


## Data Loading

In [ ]:
class Train_Dataset(Dataset):
    def __init__(self, data_path,labels, transform = None):
        self.data_path = data_path
        self.transform = transform
        self.labels = labels
    def __len__(self):
        return len(self.data_path)
    def __getitem__(self,idx):
        path = self.data_path[idx]
        img = Image.open(path)
        img = np.array(img)
        img= np.stack((img,)*3, axis=-1)
        label = self.labels[idx]
        if self.transform is not None:
            transformed = self.transform(image=img)
            image = transformed['image']
        return image, label

In [ ]:
class Val_Dataset(Dataset):
    def __init__(self, data_path,labels, transform = None):
        self.data_path = data_path
        self.transform = transform
        self.labels = labels
    def __len__(self):
        return len(self.data_path)
    def __getitem__(self,idx):
        path = self.data_path[idx]
        img = Image.open(path)
        img = np.array(img)
        img= np.stack((img,)*3, axis=-1)
        label = self.labels[idx]
        if self.transform is not None:
            transformed = self.transform(image=img)
            image = transformed['image']
        return image, label

In [ ]:
import albumentations as A                                                                           
from albumentations.pytorch import ToTensorV2

In [ ]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

data_transforms = {
    'train': A.Compose(
    [
     A.CenterCrop(384,384),
     A.RandomResizedCrop(224,224),
     A.HorizontalFlip(p=0.5),
     A.Rotate ([-7,7], p = 0.5),
     A.Normalize (p=1),
     ToTensorV2()
     ]
    ),
    'val': A.Compose([
                      A.Resize (256,256),
                      A.Normalize (p=1),
                      ToTensorV2()   
    ]
                           )
}
'''
A.RandomCrop(224, 224),
     A.OneOf([
              A.HorizontalFlip(p=1),
              A.RandomRotate90(p=1),
              A.VerticalFlip(p=1)            
    ], p=1), 
'''

'\nA.RandomCrop(224, 224),\n     A.OneOf([\n              A.HorizontalFlip(p=1),\n              A.RandomRotate90(p=1),\n              A.VerticalFlip(p=1)            \n    ], p=1), \n'

## Model

In [ ]:
pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 15.4 MB/s 


In [ ]:
import timm

In [ ]:
#timm.list_models(pretrained= False)

In [ ]:
model = timm.create_model('inception_v3',pretrained = True,num_classes = 14)

In [ ]:
model.fc = nn.Linear(in_features=2048, out_features=14, bias=True)

In [ ]:
train_data = Train_Dataset(train_path,train_labels,transform = data_transforms['train'])
val_data = Val_Dataset(val_path,val_labels,transform =  data_transforms['val'])

In [ ]:
# Top level data directory. Here we assume the format of the directory conforms
#   to the ImageFolder structure

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]

# Number of classes in the dataset
num_classes = 14
patience = 7
# Batch size for training (change depending on how much memory you have)
batch_size = 16

# Number of epochs to train for
num_epochs = 5

In [ ]:
train_data = Train_Dataset(train_path,train_labels,transform = data_transforms['train'])
val_data = Val_Dataset(val_path,val_labels,transform =  data_transforms['val'])

In [ ]:
image_datasets = {'train' : train_data , 'val' : val_data}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True) for x in ['train', 'val']}
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score , roc_curve , accuracy_score,classification_report,multilabel_confusion_matrix,auc
import tqdm

In [ ]:
from torch.types import Device
import torch.optim as optim
from torch.optim import lr_scheduler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ft = model.to(device)
criterion = nn.BCEWithLogitsLoss()

# specify optimizer
optimizer_ft = optim.Adam(model_ft.parameters(), lr=1e-4)
sgdr_partial = lr_scheduler.CosineAnnealingWarmRestarts (optimizer_ft,T_0 = 50,T_mult = 1,eta_min = 0)

In [ ]:
def train_model(model, dataloaders, criterion,  optimizer, scheduler , num_epochs=25):
    since = time.time()
    train_acc_history = []
    train_loss_hist = [] 
    train_classification_report =[]
    train_confusion_matrix = []
    val_loss_hist = []
    val_acc_history = [] 
    val_classification_report =[]
    val_confusion_matrix = []
    epoch_train_label = []
    epoch_train_output = []
    epoch_val_label = []
    epoch_val_output = []
    counting = 0
    early_stopping = False

    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_loss = 1000000000
    best_train_loss = 100000000
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            start = time.time()
            print('phase : %r' %phase)
            if phase == 'train':
                model.train() # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_acc = 0
            running_precision =0
            running_recall = 0
            running_f1_score =0 
            running_auc = 0
            # Iterate over data.
            full_label = []
            full_output = []
            full_prediction = []
            for inputs, labels in tqdm.notebook.tqdm(dataloaders[phase]):
                inputs = inputs.to(device = device, dtype = torch.float32)
                labels = labels.to(device = device)
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    outputs = model(inputs)
                    loss = criterion(outputs,labels.float())
                    pred = nn.Sigmoid()(outputs)
                    full_prediction += pred.detach().cpu().numpy().tolist()
                    pred[pred <0.5] = 0
                    pred[pred >=0.5] = 1
                    if phase == 'train':
                    # backward + optimize only if in training phase
                        loss.backward()
                        optimizer.step()
                        scheduler.step()
                # statistics
                running_loss += loss.item() * inputs.size(0)
                full_label+= labels.detach().cpu().numpy().tolist()
                full_output+= pred.detach().cpu().numpy().tolist()
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = accuracy_score(full_label,full_output)
            epoch_classification_report = classification_report(full_label,full_output,target_names = target_cols)
            epoch_multilabel_confusion_matrix = multilabel_confusion_matrix(full_label,full_output)
             
            if phase == 'train':
              train_acc_history.append(epoch_acc)
              train_loss_hist.append(epoch_loss)
              epoch_train_label.append(full_label)
              epoch_train_output.append(full_prediction)
              train_classification_report.append(epoch_classification_report)
              train_confusion_matrix.append(epoch_multilabel_confusion_matrix)
            print('\n{} Loss: {:.4f} Acc: {:.4f}, Time : {:.4f}'.format(phase, epoch_loss ,epoch_acc,time.time()-start))
            #,   , , epoch_precision,epoch_recall, epoch_auc, epoch_f1_score 
            print('\n--------------------------------')
            print('|{}Epoch {}phase multilabel_confusion_matrix report|'.format(epoch+1,phase))
            print('--------------------------------')
            print('\n{}'.format(epoch_multilabel_confusion_matrix))
            print('\n--------------------------------')
            print('\n{}Epoch {}phase classification report'.format(epoch+1,phase))
            print('--------------------------------')
            print(epoch_classification_report)

            # deep copy the model
            if phase == 'val':
              val_acc_history.append(epoch_acc)
              val_loss_hist.append(epoch_loss)
              epoch_val_label.append(full_label)
              epoch_val_output.append(full_prediction)
              val_classification_report.append(epoch_classification_report)
              val_confusion_matrix.append(multilabel_confusion_matrix)

        if phase == 'val' and epoch_loss < best_val_loss:
          best_val_loss = epoch_loss
          best_model_wts = model.state_dict()
          counting = 0
          print('Early Stopping Counter reset')
        else:
            counting += 1
            print('Early Stopping Counter : {}'.format(counting))
        if counting > 9:
          print('Early stopping!')
          early_stopping = True
          break


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best train loss : {:.4f}  Best val loss: {:4f}'.format(best_train_loss ,best_val_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    train_dict = {'Loss' : train_loss_hist , 'Accuarcy' : train_acc_history,'Classification_report' : train_classification_report,'Confusion_matrix' :train_confusion_matrix, 'train_label' : epoch_train_label , 'train_output' : epoch_train_output}
    #
    val_dict = {'Loss' :val_loss_hist , 'Accuarcy' : val_acc_history,'Classification_report' : val_classification_report,'val_label' : epoch_val_label , 'Confusion_matrix' :val_confusion_matrix,'val_output' : epoch_val_output}
    #, , 
    return model, train_dict ,val_dict,early_stopping

In [ ]:

import pickle
# Train and evaluate
start = 1
end = 50
print('\n--------------------------------')
print('|        Epoch {} ~ {}      |'.format(start, end))
print('\n--------------------------------')
model_ft, train_dict, val_dict, early_stopping= train_model(model_ft, dataloaders_dict, criterion, optimizer_ft,sgdr_partial, num_epochs=50)
torch.save(model_ft, '/content/drive/Shareddrives/캡스톤 디자인1/codes/inceptionv3_patch_size_224_random_11_multilabelclassification_{}_{}epoch.pt'.format(start, end))
with open('/content/drive/Shareddrives/캡스톤 디자인1/model_history/inceptionv3_patch_size_224_multilabelclassification_{}_{}epoch_train_dict.pkl'.format(start, end),'wb') as fw:
  pickle.dump(train_dict, fw)
with open('/content/drive/Shareddrives/캡스톤 디자인1/model_history/inceptionv3_patch_size_224_multilabelclassification_{}_{}epoch_val_dict.pkl'.format(start, end),'wb') as ff:
  pickle.dump(val_dict, ff)


'''

confusion maxrix
[[true negative , false positive
false negative , true positive]]

[[tn , fp
fn , tp]]

'''


--------------------------------
|        Epoch 1 ~ 50      |

--------------------------------
Epoch 1/50
----------
phase : 'train'


  0%|          | 0/2278 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



train Loss: 0.2925 Acc: 0.0558, Time : 380.4738

--------------------------------
|1Epoch trainphase multilabel_confusion_matrix report|
--------------------------------

[[[34465    21]
  [ 1932    20]]

 [[36179    93]
  [  166     0]]

 [[20103  2393]
  [11171  2771]]

 [[31940    42]
  [ 4448     8]]

 [[34687    10]
  [ 1741     0]]

 [[26233   808]
  [ 8232  1165]]

 [[27777   460]
  [ 7717   484]]

 [[34049     3]
  [ 2386     0]]

 [[32605   113]
  [ 3563   157]]

 [[32354    41]
  [ 4029    14]]

 [[35248     2]
  [ 1188     0]]

 [[33059    46]
  [ 3319    14]]

 [[34797    19]
  [ 1621     1]]

 [[35373    44]
  [ 1021     0]]]

--------------------------------

1Epoch trainphase classification report
--------------------------------
                    precision    recall  f1-score   support

      Cardiomegaly       0.49      0.01      0.02      1952
            Hernia       0.00      0.00      0.00       166
      Infiltration       0.54      0.20      0.29     13942
   

  0%|          | 0/647 [00:00<?, ?it/s]

...


train Loss: 0.2320 Acc: 0.2250, Time : 402.0467

--------------------------------
|29Epoch trainphase multilabel_confusion_matrix report|
--------------------------------

[[[34214   272]
  [ 1311   641]]

 [[36256    16]
  [   98    68]]

 [[18949  3547]
  [ 7529  6413]]

 [[31573   409]
  [ 3661   795]]

 [[34523   174]
  [ 1325   416]]

 [[25378  1663]
  [ 5175  4222]]

 [[26937  1300]
  [ 5571  2630]]

 [[33936   116]
  [ 2219   167]]

 [[32163   555]
  [ 2306  1414]]

 [[31940   455]
  [ 2841  1202]]

 [[35202    48]
  [ 1068   120]]

 [[32962   143]
  [ 3093   240]]

 [[34669   147]
  [ 1373   249]]

 [[35409     8]
  [ 1011    10]]]

--------------------------------

29Epoch trainphase classification report
--------------------------------
                    precision    recall  f1-score   support

      Cardiomegaly       0.70      0.33      0.45      1952
            Hernia       0.81      0.41      0.54       166
      Infiltration       0.64      0.46      0.54     13942
 

  0%|          | 0/647 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



val Loss: 0.2657 Acc: 0.1951, Time : 85.6734

--------------------------------
|29Epoch valphase multilabel_confusion_matrix report|
--------------------------------

[[[ 9732    48]
  [  463   109]]

 [[10301     3]
  [   30    18]]

 [[ 5287  1029]
  [ 2477  1559]]

 [[ 8949   142]
  [ 1048   213]]

 [[ 9784    58]
  [  383   127]]

 [[ 6915   771]
  [ 1284  1382]]

 [[ 7814   296]
  [ 1748   494]]

 [[ 9575    97]
  [  628    52]]

 [[ 9026   234]
  [  744   348]]

 [[ 9009   195]
  [  840   308]]

 [[ 9957    60]
  [  300    35]]

 [[ 9240   224]
  [  811    77]]

 [[ 9866    19]
  [  457    10]]

 [[10068    12]
  [  271     1]]]

--------------------------------

29Epoch valphase classification report
--------------------------------
                    precision    recall  f1-score   support

      Cardiomegaly       0.69      0.19      0.30       572
            Hernia       0.86      0.38      0.52        48
      Infiltration       0.60      0.39      0.47      4036
        

'\n\nconfusion maxrix\n[[true negative , false positive\nfalse negative , true positive]]\n\n[[tn , fp\nfn , tp]]\n\n'

In [ ]:
class Tunning_Dataset(Dataset):
    def __init__(self, data_path,labels, transform = None):
        self.data_path = data_path
        self.transform = transform
        self.labels = labels
    def __len__(self):
        return len(self.data_path)
    def __getitem__(self,idx):
        path = self.data_path[idx]
        img = Image.open(path)
        img = np.array(img)
        img= np.stack((img,)*3, axis=-1)
        label = self.labels[idx]
        if self.transform is not None:
            transformed = self.transform(image=img)
            image = transformed['image']
        return image, label

In [ ]:
data_transforms = {
    'train': A.Compose(
    [
     A.Resize(299,299),
     A.HorizontalFlip(p=0.5),
     A.Rotate ([-7,7], p = 0.5),
     A.Normalize (p=1),
     ToTensorV2()
     ]
    ),
    'val': A.Compose([
                      A.Resize (299,299),
                      A.Normalize (p=1),
                      ToTensorV2()   
    ]
                           )
}
'''
A.RandomCrop(224, 224),
     A.OneOf([
              A.HorizontalFlip(p=1),
              A.RandomRotate90(p=1),
              A.VerticalFlip(p=1)            
    ], p=1), 
'''

'\nA.RandomCrop(224, 224),\n     A.OneOf([\n              A.HorizontalFlip(p=1),\n              A.RandomRotate90(p=1),\n              A.VerticalFlip(p=1)            \n    ], p=1), \n'

In [ ]:
tunning_data = Tunning_Dataset(tuning_path,tuning_labels,transform = data_transforms['train'])
val_data = Val_Dataset(val_path,val_labels,transform =  data_transforms['val'])

In [ ]:
image_datasets = {'train' : tunning_data , 'val' : val_data}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True) for x in ['train', 'val']}
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
from torch.types import Device
import torch.optim as optim
from torch.optim import lr_scheduler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.BCEWithLogitsLoss()

# specify optimizer
optimizer_ft = optim.Adam(model_ft.parameters(), lr=1e-4)
sgdr_partial = lr_scheduler.CosineAnnealingWarmRestarts (optimizer_ft,T_0 = 50,T_mult = 1,eta_min = 0)

In [ ]:

import pickle
# Train and evaluate
start = 1
end = 50
print('\n--------------------------------')
print('|        Epoch {} ~ {}      |'.format(start, end))
print('\n--------------------------------')
model_ft, train_dict, val_dict, early_stopping= train_model(model_ft, dataloaders_dict, criterion, optimizer_ft,sgdr_partial, num_epochs=50)
torch.save(model_ft, '/content/drive/Shareddrives/캡스톤 디자인1/codes/inceptionv3_patch_size_224_multilabelclassification_tunning_{}_{}epoch.pt'.format(start, end))
with open('/content/drive/Shareddrives/캡스톤 디자인1/model_history/inceptionv3_patch_size_224_multilabelclassification_tunning_{}_{}epoch_train_dict.pkl'.format(start, end),'wb') as fw:
  pickle.dump(train_dict, fw)
with open('/content/drive/Shareddrives/캡스톤 디자인1/model_history/inceptionv3_patch_size_224_multilabelclassification_tunning_{}_{}epoch_val_dict.pkl'.format(start, end),'wb') as ff:
  pickle.dump(val_dict, ff)



'''

confusion maxrix
[[true negative , false positive
false negative , true positive]]

[[tn , fp
fn , tp]]

'''


--------------------------------
|        Epoch 1 ~ 50      |

--------------------------------
Epoch 1/50
----------
phase : 'train'


  0%|          | 0/311 [00:00<?, ?it/s]


train Loss: 0.2530 Acc: 0.1940, Time : 85.5290

--------------------------------
|1Epoch trainphase multilabel_confusion_matrix report|
--------------------------------

[[[4681   36]
  [ 202   50]]

 [[4956    0]
  [  11    2]]

 [[2525  528]
  [1128  788]]

 [[4301   54]
  [ 553   61]]

 [[4682   22]
  [ 217   48]]

 [[3461  254]
  [ 705  549]]

 [[3703  150]
  [ 826  290]]

 [[4645    5]
  [ 315    4]]

 [[4418   61]
  [ 368  122]]

 [[4314   64]
  [ 453  138]]

 [[4802    4]
  [ 160    3]]

 [[4521    2]
  [ 444    2]]

 [[4748    7]
  [ 212    2]]

 [[4831    0]
  [ 138    0]]]

--------------------------------

1Epoch trainphase classification report
--------------------------------
                    precision    recall  f1-score   support

      Cardiomegaly       0.58      0.20      0.30       252
            Hernia       1.00      0.15      0.27        13
      Infiltration       0.60      0.41      0.49      1916
            Nodule       0.53      0.10      0.17       614


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


  0%|          | 0/647 [00:00<?, ?it/s]

...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


  0%|          | 0/647 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



val Loss: 0.3652 Acc: 0.1997, Time : 88.8595

--------------------------------
|11Epoch valphase multilabel_confusion_matrix report|
--------------------------------

[[[ 9688    92]
  [  451   121]]

 [[10299     5]
  [   35    13]]

 [[ 4589  1727]
  [ 1985  2051]]

 [[ 8622   469]
  [  919   342]]

 [[ 9639   203]
  [  299   211]]

 [[ 6861   825]
  [ 1320  1346]]

 [[ 6975  1135]
  [ 1265   977]]

 [[ 9301   371]
  [  552   128]]

 [[ 8987   273]
  [  730   362]]

 [[ 8886   318]
  [  735   413]]

 [[ 9910   107]
  [  285    50]]

 [[ 9078   386]
  [  774   114]]

 [[ 9653   232]
  [  364   103]]

 [[10025    55]
  [  265     7]]]

--------------------------------

11Epoch valphase classification report
--------------------------------
                    precision    recall  f1-score   support

      Cardiomegaly       0.57      0.21      0.31       572
            Hernia       0.72      0.27      0.39        48
      Infiltration       0.54      0.51      0.52      4036
        

'\n\nconfusion maxrix\n[[true negative , false positive\nfalse negative , true positive]]\n\n[[tn , fp\nfn , tp]]\n\n'